In [0]:
#https://github.com/talhaanwarch/DeftEval2020
#https://www.aclweb.org/anthology/C18-1247.pdf how emotional are you
#https://arxiv.org/pdf/1905.12516.pdf multiple datasets 
#In recent years there has been exponential growth in usage of social media web sites such as Twitter, Facebook, and Reddit. It has led to an urgent need to tackle hate speech and offensive language on these platforms. Such language may induce differences between different groups, which may further cause an imbalance in society. In this paper, we propose a Deep Learning-based model using the Attention mechanism and Bi-directional LSTM(Long Short Term Memory) to classify among hate, offensive, and neutral speech. We conducted various experiments on data preprocessing and tried on different models to achieve new state-of-the-art results (as per our knowledge). We have trained and tested our models on annotated data by Davidson. Et al., 2017. We have evaluated our best model on various metrics such as F1-score, precision, recall, and accuracy. With our proposed model, we achieved an F1-score of 0.96, recall of 0.97, the precision of 0.97, and accuracy of 0.97, which is a weighted average of three classes.

In [0]:
import pandas as pd
import numpy as np
import json
import keras
import tensorflow as tf
import io
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import Conv1D, Conv2D
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import LabelEncoder
import nltk
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.layers import average
import tensorflow_hub as hub
from keras.layers import Average
from keras.layers import Concatenate
nltk.download('punkt')
from numpy import random
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
#plt.switch_backend('agg')
%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install keras_metrics
!pip install keras-self-attention
import keras_metrics as km

  Created wheel for keras-self-attention: filename=keras_self_attention-0.42.0-cp36-none-any.whl size=17296 sha256=e03e986307249ff13d125cb65feca981529e7a6ba8aae6566fac8b8b0e3c88ba
  Stored in directory: /root/.cache/pip/wheels/7b/05/a0/99c0cf60d383f0494e10eca2b238ea98faca9a1fe03cac2894
Successfully built keras-self-attention


In [0]:
root_path='/content/drive/My Drive/offenseval/'

In [0]:
train_data=pd.read_csv('/content/drive/My Drive/offenseval/OLIDv1.0/olid-training-v1.0.tsv',delimiter='\t')
test_levela=pd.read_csv('/content/drive/My Drive/offenseval/OLIDv1.0/testset-levelc.tsv',delimiter='\t')
labels_levels=pd.read_csv('/content/drive/My Drive/offenseval/OLIDv1.0/labels-levelc.csv',header=None)

In [0]:
test_data=pd.read_csv('/content/drive/My Drive/offenseval/OLIDv1.0/testset-levelc.tsv',delimiter='\t')

In [0]:
print(train_data.head())

      id                                              tweet  ... subtask_b subtask_c
0  86426  @USER She should ask a few native Americans wh...  ...       UNT       NaN
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...  ...       TIN       IND
2  16820  Amazon is investigating Chinese employees who ...  ...       NaN       NaN
3  62688  @USER Someone should'veTaken" this piece of sh...  ...       UNT       NaN
4  43605  @USER @USER Obama wanted liberals &amp; illega...  ...       NaN       NaN

[5 rows x 5 columns]


In [0]:
print(test_data.head())

      id                                              tweet
0  15923  #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1  60133  #NoPasaran: Unity demo to oppose the far-right...
2  83681           . . . What the fuck did he do this time?
3  65507  @USER Do you get the feeling he is kissing @US...
4  34263  #StopKavanaugh he is liar like the rest of the...


In [0]:
print(labels_levels.head())

       0    1
0  15923  OTH
1  60133  GRP
2  83681  IND
3  65507  OTH
4  34263  IND


In [0]:
train_data_1=train_data[train_data['subtask_b']=="TIN"].reset_index(drop=True)

In [0]:
labels_train_raw=list(train_data_1['subtask_c'])
labels_testa=labels_levels[1]

In [0]:
le=LabelEncoder()
le.fit(labels_train_raw)
labels_train_le=le.transform(labels_train_raw)
labels_dev_le=le.transform(labels_testa)

In [0]:
ohc=OneHotEncoder()

In [0]:
ohc=OneHotEncoder()
labels_train=ohc.fit_transform(labels_train_le.reshape(-1,1))
labels_dev=ohc.transform(labels_dev_le.reshape(-1,1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## load embeddings

In [0]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Sentimix/word2vec_twitter_tokens.bin', binary=True,unicode_errors='ignore')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model['fuck'].shape

(400,)

In [0]:
# !pip install fasttext
# import fasttext
# import os
# root_path = "/content/drive/My Drive/Sentimix"
# # model = fasttext.load_model(os.path.join(root_path,'fasttext_twitter_raw.bin'))
# model = fasttext.load_model(os.path.join(root_path,'fasttext_twitter_raw.bin'))

# # model = fasttext.load_model("/content/drive/My Drive/Sentimix/fasttext_twitter_raw.bin")

# word_inv = 'running'
# word_oov = 'runnnnnnnnnnnning'

# if word_oov in model:
#     print('This word is in the vocabulary: {}'.format(word_oov))
# else:
#     print('This word is NOT in the vocabulary: {}'.format(word_oov))

## cleaning

In [0]:
le.classes_

array(['GRP', 'IND', 'OTH'], dtype='<U3')

In [0]:
labels_train_raw[12]

'GRP'

In [0]:
train_data_1.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
1,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH
2,52415,@USER was literally just talking about this lo...,OFF,TIN,GRP
3,13384,@USER Canada doesn’t need another CUCK! We alr...,OFF,TIN,IND
4,28414,@USER you are a lying corrupt traitor!!! Nobod...,OFF,TIN,IND


In [0]:
def remove_pattern(input_txt, pattern,with_space=False):
    r = re.findall(pattern, input_txt)
    if with_space==False:
      for i in r:
        input_txt = re.sub(i, '', input_txt)
    else:
      for i in r:
        input_txt = re.sub(i, ' ', input_txt)
    return input_txt 
!pip install emoji
import emoji
import pickle
import re
with open('/content/drive/My Drive/Sentimix/helper_data/contractions.pkl','rb')as f:
  contractions=pickle.load(f)


from collections import Counter
contractions=Counter(contractions)
with open('/content/drive/My Drive/Sentimix/helper_data/acronyms.pkl','rb')as f:
  acronyms=pickle.load(f)
acronyms=Counter(acronyms)
def acronym(df,column):
  s_l=[]
  for i in range(df.shape[0]):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if acronyms[word]!=0:
        w_l.append(acronyms[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
with open('/content/drive/My Drive/Sentimix/hinglish_to_english.pickle','rb')as f:
  hing_to_eng=pickle.load(f)
hing_to_eng=Counter(hing_to_eng)
def hindi_se_english(df,column):
  s_l=[]
  for i in range(df.shape[0]):
    w_l=[]
    sent=str(df[column][i])
    for word in sent.split():
      if hing_to_eng[word]!=0:
        w_l.append(hing_to_eng[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
with open('/content/drive/My Drive/Sentimix/Hinglish_utils/Hinglish_Profanity_dict.pkl', 'rb') as handle:
    cuss_dict=pickle.load(handle)
cuss_dict=Counter(cuss_dict)
def replace_cuss(df,column):
  s_l=[]
  for i in range(df.shape[0]):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if cuss_dict[word]!=0:
        w_l.append('abuse')
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_contraction(df,column):
  s_l=[]
  for i in range(df.shape[0]):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if contractions[word]!=0:
        w_l.append(contractions[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_pattern_rep(input_txt, pattern,rep_pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
      input_txt = re.sub(i, rep_pattern, input_txt)

    return input_txt 
def cleaning(data_f,cleaning_col,new_col):
  for i in range(data_f.shape[0]):
    data_f[cleaning_col][i]=emoji.demojize(str(data_f[cleaning_col][i]))
  #data_f[cleaning_col]=replace_cuss(data_f,cleaning_col)
  data_f[new_col]=np.vectorize(remove_pattern)(data_f[cleaning_col],"_",with_space=True)
  data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],"-",with_space=True)
  data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],":",with_space=True)
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "@[\w]*"," ")
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "http\S+"," ")
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "[0-9]+"," ")
  #data_f[new_col]=hindi_se_english(data_f,cleaning_col)
  data_f[new_col]=remove_contraction(data_f,new_col)
  data_f[new_col]=acronym(data_f,new_col)
  data_f[new_col]=data_f[new_col].str.replace("[^a-zA-Z]<>", " ")
  data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "~",with_space=False)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "!",with_space=True)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], ".",with_space=True)
  #data_f[new_col] = data_f[new_col].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
  return data_f
import numpy as np
a=cleaning(train_data_1,'tweet','clean_col')


     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=ac420bb951897aa0d154a4e1cbe87ad20b466299071bdc0dcd83d8495889a1dd
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
b=cleaning(test_data,'tweet','clean_col')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Tokenization

In [0]:
contractions["can't"]

'cannot'

In [0]:
print(train_data_1.columns)

Index(['id', 'tweet', 'subtask_a', 'subtask_b', 'subtask_c', 'clean_col'], dtype='object')


In [0]:
max_words =5000
max_len = 25
tok = Tokenizer(max_words)
tok.fit_on_texts(a['clean_col'].astype(str))


In [0]:
sequences_train = tok.texts_to_sequences(a['clean_col'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
sequences_dev = tok.texts_to_sequences(b['clean_col'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [0]:
labels_train_raw[9]

'IND'

In [0]:
sequences_train[1]

[21, 8, 27, 4390]

In [0]:
sequences_matrix_train[1]

array([  21,    8,   27, 4390,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int32)

In [0]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})


## abuse feature

In [0]:
abuse_f=np.zeros((train_data.shape[0]))

In [0]:
for i in range(train_data.shape[0]):
  sent=str(a['new_col'][i])
  for word in sent.split():
    if word=='abuse':
      abuse_f[i]+=1

In [0]:
abuse_f[:10]

array([0., 3., 0., 0., 0., 3., 0., 0., 0., 1.])

In [0]:
abuse_fd=np.zeros((dev_data.shape[0]))


In [0]:
for i in range(dev_data.shape[0]):
  sent=str(b['new_col'][i])
  for word in sent.split():
    if word=='abuse':
      abuse_fd[i]+=1

## CNN

In [0]:
def cnn(embed_size):
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(len(tok.word_index) + 1,embed_size,weights=[embedding_matrix_1],
                  trainable=True)(inputs)
    #layer = Embedding(vocab_size,100,input_length=max_len)(inputs)
    x = Conv1D(256, 3, activation='relu')(layer)
    x = MaxPooling1D(3)(x)

    x = Conv1D(128, 4, activation='relu')(x)
    x = LSTM(100,recurrent_dropout=0.2)(x)
    layer = Dense(200,name='FC1')(x)
    layer = BatchNormalization(name = 'BN1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(300,name='FC2')(layer)
    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = BatchNormalization(name = 'BN4')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [0]:
model_cnn=cnn(embed_size=300)

In [0]:
model_cnn.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])

In [0]:
model_cnn.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 20)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 20, 300)           5891100   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 18, 256)           230656    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 256)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3, 128)            131200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               91600     
_________________________________________________________________
FC1 (Dense)                  (None, 200)               2020

In [0]:
model_cnn.fit(sequences_matrix_train,labels_train_le,validation_data=(sequences_matrix_dev,labels_dev_le),epochs=5,batch_size=32,callbacks=[cp_check_point,reduce_lr,f1_metrics])

Train on 13240 samples, validate on 860 samples
Epoch 1/5
13240/13240 [==============================] - 14s 1ms/step - loss: 0.6090 - acc: 0.6693 - val_loss: 0.5511 - val_acc: 0.7337
 — val_f1: 0.698211 — val_precision: 0.691172 — val_recall 0.723387
Epoch 2/5
13240/13240 [==============================] - 9s 686us/step - loss: 0.4735 - acc: 0.8024 - val_loss: 0.4552 - val_acc: 0.7965
 — val_f1: 0.736279 — val_precision: 0.748642 — val_recall 0.727352
Epoch 3/5
13240/13240 [==============================] - 9s 658us/step - loss: 0.3598 - acc: 0.8738 - val_loss: 0.4840 - val_acc: 0.7744
 — val_f1: 0.715259 — val_precision: 0.718951 — val_recall 0.712030
Epoch 4/5
13240/13240 [==============================] - 9s 666us/step - loss: 0.2601 - acc: 0.9259 - val_loss: 0.5610 - val_acc: 0.7395
 — val_f1: 0.682690 — val_precision: 0.679532 — val_recall 0.686559
Epoch 5/5
13240/13240 [==============================] - 9s 658us/step - loss: 0.1967 - acc: 0.9518 - val_loss: 0.5775 - val_acc: 0.7

## RNN

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,200,input_length=max_len)(inputs)
    layer = LSTM(100,recurrent_dropout=0.2)(layer)
    layer = Dense(200,name='FC1')(layer)
    layer = BatchNormalization(name = 'BN1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(300,name='FC2')(layer)
    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(3,name='out_layer')(layer)
    layer = BatchNormalization(name = 'BN4')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [0]:
model=RNN()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['acc'])

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 25)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 200)           3000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
FC1 (Dense)                  (None, 200)               20200     
_________________________________________________________________
BN1 (BatchNormalization)     (None, 200)               800       
_________________________________________________________________
activation_2 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0   

In [0]:
model.fit(sequences_matrix_train,labels_train,validation_data=(sequences_matrix_dev,labels_dev),epochs=5,batch_size=32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 15131 samples, validate on 1869 samples
Epoch 1/5





15131/15131 [==============================] - 35s 2ms/step - loss: 1.0150 - acc: 0.4766 - val_loss: 0.9824 - val_acc: 0.5409
Epoch 2/5
15131/15131 [==============================] - 28s 2ms/step - loss: 0.8010 - acc: 0.6447 - val_loss: 0.9687 - val_acc: 0.5538
Epoch 3/5
15131/15131 [==============================] - 28s 2ms/step - loss: 0.6731 - acc: 0.7356 - val_loss: 1.1291 - val_acc: 0.5431
Epoch 4/5
15131/15131 [==============================] - 27s 2ms/step - loss: 0.5667 - acc: 0.8005 - val_loss: 1.1341 - val_acc: 0.5522
Epoch 5/5
 4544/15131 [========>.....................] - ETA: 18s - loss: 0.4483 - acc: 0.8638

KeyboardInterrupt: ignored

In [0]:
train_data['sent'][1]

'Madarchod mulle ye mathura me Nahi dikha tha jab mullo ne Hindu ko iss liye mara ki vo lasse ki paise mag liye the       t   co   oxf tr bly '

## combinin train and dev

In [0]:
sequences_matrix_dev.shape

(1869, 20)

In [0]:
new_mat=np.concatenate((sequences_matrix_train,sequences_matrix_dev))

In [0]:
new_mat.shape

(20189, 20)

In [0]:
labels_train.shape

(18320, 3)

In [0]:
labels_dev.shape

(1869, 3)

In [0]:
labels_train

<18320x3 sparse matrix of type '<class 'numpy.float64'>'
	with 18320 stored elements in Compressed Sparse Row format>

In [0]:
new_label=np.concatenate((np.array(labels_train),np.array(labels_dev)),axis=0)
print(new_label.shape)

(20189,)


In [0]:
ohc=OneHotEncoder()
new_label=ohc.fit_transform(new_label.reshape(-1,1))


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
new_abuse=np.concatenate((abuse_f,abuse_fd))
print(new_abuse.shape)

(20189,)


## Attention Network

In [0]:
# labels_dev.shape

In [0]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []
  
  def on_epoch_end(self, epoch, logs={}):
    val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
    val_targ = self.validation_data[1]
    #val_predict=val_predict.argmax(axis=-1)
    
    #val_targ=val_targ.argmax(axis=-1)
    # print(val_predict)
    # print(val_targ)
    _val_f1 = f1_score(val_targ, val_predict,average='macro')
    _val_recall = recall_score(val_targ, val_predict,average='macro')
    _val_precision = precision_score(val_targ, val_predict,average='macro')
    self.val_f1s.append(_val_f1)
    self.val_recalls.append(_val_recall)
    self.val_precisions.append(_val_precision)
    print(" — val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
    return
 
f1_metrics = Metrics()

In [0]:
embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/IR_project/glove.6B', 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
word_index=tok.word_index

In [0]:
embedding_matrix_4 = np.zeros((len(tok.word_index) + 1, 400))
for word, i in tok.word_index.items():
    if word in model:
      embedding_matrix_4[i] = model[word]

In [0]:
embedding_matrix_1 = np.zeros((len(tok.word_index) + 1, 300))
for word, i in tok.word_index.items():
    if word in embeddings_index.keys():
      embedding_matrix_1[i] = embeddings_index[word]

In [0]:
from keras.layers import Concatenate

In [0]:
from sklearn.metrics import precision_recall_fscore_support

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
from keras_self_attention import SeqSelfAttention
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    #x1=Embedding(len(word_index)+1,embed_size)(inp1)
    x1 = Embedding(len(tok.word_index) + 1,embed_size,weights=[embedding_matrix_4],
                  trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    # x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    x1=SpatialDropout1D(0.1)(x1)
    x1 = CuDNNLSTM(400, return_sequences=True)(x1)  
    # x1 = SeqSelfAttention(kernel_regularizer=keras.regularizers.l2(1e-4),
    #                    bias_regularizer=keras.regularizers.l1(1e-4),
    #                    attention_regularizer_weight=1e-4,
    #                    name='Attention')(x1)  
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(3,name='out_layer',activation='softmax')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=400)

In [0]:
model_bi.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['acc',km.f1_score()])

In [0]:
model_bi.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 25, 400)      3920800     input_12[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_12 (SpatialDr (None, 25, 400)      0           embedding_12[0][0]               
__________________________________________________________________________________________________
cu_dnnlstm_12 (CuDNNLSTM)       (None, 25, 400)      1283200     spatial_dropout1d_12[0][0]       
___________________________________________________________________________________________

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels_train_le),labels_train_le)
class_weights=dict(enumerate(class_weights))

In [0]:
print(class_weights)

{0: 1.2029795158286778, 1: 0.5367677606979643, 2: 3.270886075949367}


In [0]:
cp_filepath=root_path+'checkpoints/lstm_model_task3.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)


In [0]:
model_bi.fit([sequences_matrix_train],labels_train_le,validation_data=([sequences_matrix_dev],labels_dev_le),class_weight=class_weights,epochs=5,batch_size=32,callbacks=[cp_check_point])

Train on 3876 samples, validate on 213 samples
Epoch 1/5
3876/3876 [==============================] - 1s 351us/step - loss: 0.0708 - acc: 0.9703 - f1_score: 0.0270 - val_loss: 2.5237 - val_acc: 0.6620 - val_f1_score: 0.4387
Epoch 2/5
3876/3876 [==============================] - 1s 358us/step - loss: 0.1198 - acc: 0.9564 - f1_score: 0.0363 - val_loss: 2.2011 - val_acc: 0.6573 - val_f1_score: 0.3938
Epoch 3/5
3876/3876 [==============================] - 1s 373us/step - loss: 0.0559 - acc: 0.9786 - f1_score: 0.0177 - val_loss: 2.7954 - val_acc: 0.5540 - val_f1_score: 0.1692
Epoch 4/5
3876/3876 [==============================] - 1s 382us/step - loss: 0.0506 - acc: 0.9830 - f1_score: 0.0126 - val_loss: 2.5595 - val_acc: 0.6385 - val_f1_score: 0.4286
Epoch 5/5
3876/3876 [==============================] - 1s 366us/step - loss: 0.0946 - acc: 0.9706 - f1_score: 0.0238 - val_loss: 1.9843 - val_acc: 0.6385 - val_f1_score: 0.3858


In [0]:
model_json = model.to_json()
with open(root_path+"model_attention_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(root_path+"model_attention_1.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from keras.models import model_from_json

In [0]:
json_file = open(root_path+"model_attention_1.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects={'Attention':Attention})
# load weights into new model
loaded_model.load_weights(root_path+"model_attention_1.h5")

In [0]:
y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [0]:
from sklearn.metrics import classification_report

#y_pred = model_bi.predict(X_dev, batch_size=30, verbose=1)

print(classification_report(labels_dev_le, np.argmax(y_preds_dl,axis=-1),labels=[0,1,2]))

              precision    recall  f1-score   support

           0       0.60      0.65      0.63        78
           1       0.71      0.75      0.73       100
           2       0.43      0.29      0.34        35

    accuracy                           0.64       213
   macro avg       0.58      0.56      0.57       213
weighted avg       0.63      0.64      0.63       213



In [0]:
from sklearn.metrics import f1_score
f1_score(labels_dev_le, np.argmax(y_preds_dl,axis=-1),labels=[0,1,2],average='macro')

0.5627030898301616

## Transfer Learning

In [0]:
from keras.models import model_from_json

In [0]:

with open("/content/drive/My Drive/Sentimix/Abhishek Folder/transfer_model.json") as json_file:
  model = model_from_json(json_file.read(),custom_objects={'Attention': Attention})
  model.load_weights("/content/drive/My Drive/Sentimix/Abhishek Folder/transfer_model.h5")

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 300)           7346700   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 400)           801600    
_________________________________________________________________
attention_1 (Attention)      (None, 400)               420       
_________________________________________________________________
FC1 (Dense)                  (None, 200)               80200     
_________________________________________________________________
activation_2 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0   

In [0]:
for layer in model.layers[:-3]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7ff8f87edc18> False
<keras.layers.embeddings.Embedding object at 0x7ff8f87edc88> False
<keras.layers.wrappers.Bidirectional object at 0x7ff8fb2975f8> False
<__main__.Attention object at 0x7ff8fb2c0048> False
<keras.layers.core.Dense object at 0x7ff8fb2c0080> False
<keras.layers.core.Activation object at 0x7ff8fb2c00b8> False
<keras.layers.core.Dropout object at 0x7ff8fb2c0278> False
<keras.layers.core.Dense object at 0x7ff8fb2c02b0> False
<keras.layers.core.Activation object at 0x7ff8fb2c02e8> True
<keras.layers.core.Dropout object at 0x7ff8fb2c0438> True
<keras.layers.core.Dense object at 0x7ff8fb2c0470> True


In [0]:
model.layers[5].output

<tf.Tensor 'activation_2_1/Relu:0' shape=(?, 200) dtype=float32>

In [0]:
layer = model.layers[5].output
layer = Dense(32,activation=custom_gelu)(layer)
layer = Dense(64,activation=custom_gelu)(layer)
layer = Dense(3,activation="softmax")(layer)

new_model = Model(inputs=model.input,outputs=layer)

In [0]:
new_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 300)           7346700   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 400)           801600    
_________________________________________________________________
attention_1 (Attention)      (None, 400)               420       
_________________________________________________________________
FC1 (Dense)                  (None, 200)               80200     
_________________________________________________________________
activation_2 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                6432

In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
new_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['acc',f1])

In [0]:
new_model.fit(sequences_matrix_train,labels_train,validation_data=(sequences_matrix_dev,labels_dev),epochs=5,batch_size=32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 15131 samples, validate on 1869 samples
Epoch 1/5
15131/15131 [==============================] - 25s 2ms/step - loss: 1.0892 - acc: 0.3780 - f1: 0.0034 - val_loss: 1.0784 - val_acc: 0.4147 - val_f1: 0.0000e+00
Epoch 2/5
15131/15131 [==============================] - 23s 1ms/step - loss: 1.0755 - acc: 0.3957 - f1: 0.0246 - val_loss: 1.0784 - val_acc: 0.4114 - val_f1: 0.0851
Epoch 3/5
15131/15131 [==============================] - 22s 1ms/step - loss: 1.0675 - acc: 0.4077 - f1: 0.0637 - val_loss: 1.0622 - val_acc: 0.4136 - val_f1: 0.0722
Epoch 4/5
15131/15131 [==============================] - 22s 1ms/step - loss: 1.0598 - acc: 0.4180 - f1: 0.0881 - val_loss: 1.0676 - val_acc: 0.4034 - val_f1: 0.0973
Epoch 5/5
15131/15131 [==============================] - 22s 1ms/step - loss: 1.0556 - acc: 0.4202 - f1: 0.1087 - val_loss: 1.0620 - val_acc: 0.4082 - val_f1: 0.0899


In [0]:
s='madarchod ho tum'
s_seq = tok.texts_to_sequences([s])
s_seq = sequence.pad_sequences(s_seq,maxlen=max_len,padding='post',truncating='post')

In [0]:
model.predict([s_seq])

array([[0.45833895, 0.48547915, 0.05618191]], dtype=float32)

In [0]:
s_seq


array([[1001,   16,   72,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

## TF-IDF and count features

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?' ]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
(4, 9)


In [0]:
# train_data=pd.read_csv('/content/drive/My Drive/Sentimix/train_clean_with_cusswords_1.csv')
# dev_data=pd.read_csv('/content/drive/My Drive/Sentimix/dev_clean_with_cusswords_1.csv')

In [0]:
vectorizer =TfidfVectorizer(max_features=5000,lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,5),dtype=np.float32,max_df=0.5,min_df=2)
vectorizer.fit(train_data_1['clean_col'].astype(str))
x_train=vectorizer.transform(train_data_1['clean_col'].astype(str))
x_dev=vectorizer.transform(test_data['clean_col'].astype(str))

In [0]:
# vectorizer_char =TfidfVectorizer(max_features=20000, lowercase=True, analyzer='char',ngram_range=(1,5),dtype=np.float32,max_df=0.5,min_df=8)
# vectorizer_char.fit(train_data['clean_col'].astype(str))
# x_train_char=vectorizer_char.transform(train_data['clean_col'].astype(str))
# x_dev_char=vectorizer_char.transform(dev_data['clean_col'].astype(str))

In [0]:
x_train.shape

(3876, 5000)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_data_1['clean_col'])
X_dev_counts=count_vect.transform(test_data['clean_col'])
X_train_counts.shape

(3876, 9364)

## XGBoost

In [0]:
import xgboost as xgb

In [0]:
xgb_params = {'learning_rate': 0.05, 
              'max_depth': 10,
              'subsample': 0.9,        
              'colsample_bytree': 0.9,
              'objective': 'binary:logistic',
              'silent': 1, 
              'n_estimators':50, 
              'gamma':1,         
              'min_child_weight':4}   
clf = xgb.XGBClassifier(**xgb_params, seed = 10)

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
le_x=LabelEncoder()
label_train_le=le_x.fit_transform(labels_train_raw)
label_dev_le=le_x.fit_transform(labels_dev_raw)

In [0]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=42)
results = cross_val_score(clf, x_train, label_train_le, cv=skf)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 49.89% (1.39%)


In [0]:
print(results)

[0.5794516  0.6316485  0.61413941 0.56576338 0.57506614]


In [0]:
clf.fit(x_train,label_train_le)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=1,
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=4, missing=None, n_estimators=50, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=10, silent=1,
              subsample=0.9, verbosity=1)

In [0]:
y_preds=clf.predict(x_dev)

In [0]:
y_preds

array([2, 2, 2, ..., 2, 2, 2])

In [0]:
from sklearn.metrics import f1_score
f1_score(label_dev_le,y_preds,average='weighted')

0.3417017593203973

## Baseline models

In [0]:
from sklearn.metrics import f1_score

In [0]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, labels_train_le)
y_preds=clf.predict(X_dev_counts)
f1_score(labels_dev_le,y_preds,average='macro')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4728778467908903

In [0]:
from sklearn.linear_model import SGDClassifier
sgd_clf=SGDClassifier().fit(X_train_counts,labels_train_le)
y_preds=sgd_clf.predict(X_dev_counts)
f1_score(labels_dev_le,y_preds,average='macro')

0.5132202152891807

In [0]:
le.classes_

array(['GRP', 'IND', 'OTH'], dtype='<U3')

In [0]:
from sklearn.svm import SVC
clf_svc=SVC(gamma='scale',decision_function_shape='ovo',C=10).fit(X_train_counts,labels_train_le)
y_preds=clf_svc.predict(X_dev_counts)
f1_score(labels_dev_le,y_preds,average='macro')

0.45956858319159427

In [0]:
f1_score(labels_dev_le,y_preds_dl,average='macro')

0.9868242741878198

## Text Blob

In [0]:
import textblob

In [0]:
from textblob import TextBlob

In [0]:
blob = TextBlob(train_data['tidy_tweet_abuse_1'][19])

In [0]:
train_data['tidy_tweet_abuse_1'][10:20]

10    omer muhammadpti pakhtoon haqooq jang other th...
11    those sale have state ever sage other than sak...
12    baby sorry wasn online"  hope sleeping well th...
13    straightforward"  life sneak diye hain mene al...
14    mohsin dawartweets daikho a tad"  bayghayrat i...
15    a tad"  media channel dikh rahe they" up form ...
16    malum ammi chuda bsdk grahm stains dara singh ...
17         well birdseed sneak chele thirty"  news year
18    twitter baghair their roza mumkin other than h...
19    haha soldier"  reading american gods hardy eac...
Name: tidy_tweet_abuse_1, dtype: object

In [0]:
train_data['sent'][19]

'oh haha the soldier is reading american gods hardy each"  each"  thanks for reminding everyone of your other clusterfuck '

In [0]:
print(blob.sentences[0])

haha soldier"  reading american gods hardy each each thanks reminding everyone your other clusterfuck


In [0]:
blob.sentences[0].sentiment.polarity

0.06875

## Charcter level LSTM

### charcter tokenization

In [0]:
max_words = 15000
max_len = 20
tok_char = Tokenizer(
    char_level=True,
    filters=None,
    lower=False,
)


In [0]:
sequences_train_char = tok.texts_to_sequences(a['hindi_clean'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_train_char = sequence.pad_sequences(sequences_train_char,maxlen=110,padding='post',truncating='post')

In [0]:
sequences_dev_char = tok.texts_to_sequences(b['hindi_clean'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_dev_char = sequence.pad_sequences(sequences_dev_char,maxlen=110,padding='post',truncating='post')

In [0]:
model=BidLstm(110,max_features=max_words,embed_size=300)

In [0]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['acc'])

In [0]:
model.fit([sequences_matrix_train_char],labels_train,validation_data=([sequences_matrix_dev_char],labels_dev),epochs=5,batch_size=32)

Train on 15131 samples, validate on 1869 samples
Epoch 1/5
15131/15131 [==============================] - 175s 12ms/step - loss: 1.0970 - acc: 0.3684 - val_loss: 1.0897 - val_acc: 0.4034
Epoch 2/5
15131/15131 [==============================] - 174s 11ms/step - loss: 1.0955 - acc: 0.3708 - val_loss: 1.0902 - val_acc: 0.4034
Epoch 3/5
15131/15131 [==============================] - 173s 11ms/step - loss: 1.0948 - acc: 0.3713 - val_loss: 1.0901 - val_acc: 0.4034
Epoch 4/5
15131/15131 [==============================] - 172s 11ms/step - loss: 1.0948 - acc: 0.3714 - val_loss: 1.0896 - val_acc: 0.4034
Epoch 5/5
15131/15131 [==============================] - 172s 11ms/step - loss: 1.0944 - acc: 0.3726 - val_loss: 1.0897 - val_acc: 0.4034


## Bert

In [0]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.2MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = "\"/content/gdrive/My Drive/offenseval/Bert/checkpoints/\""#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: "/content/gdrive/My Drive/defteval/Bert/checkpoints/" *****


In [0]:
DATA_COLUMN = 'clean_col'
LABEL_COLUMN = 'subtask_a'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = ['NOT','OFF']

In [0]:
train_data[LABEL_COLUMN]

0        OFF
1        OFF
2        NOT
3        OFF
4        NOT
        ... 
13235    OFF
13236    NOT
13237    OFF
13238    OFF
13239    NOT
Name: subtask_a, Length: 13240, dtype: object

In [0]:
train_df,test_df=train_test_split(train_data,test_size=0.2)

In [0]:
train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
test_InputExamples = test_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
import tensorflow_hub as hub
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
MAX_SEQ_LENGTH = 30
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 10592


INFO:tensorflow:Writing example 0 of 10592


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: NOT (id = 0)


INFO:tensorflow:label: NOT (id = 0)


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


In [0]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        #accuracy_15=tf.keras.metrics.top_k_categorical_accuracy(label_ids,predicted_labels,k=15)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
           # "accuracy_15":accuracy_15,
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
BATCH_SIZE = 10
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:


model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '"/content/gdrive/My Drive/defteval/Bert/checkpoints/"', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fc8619828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '"/content/gdrive/My Drive/defteval/Bert/checkpoints/"', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fc8619828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
from datetime import datetime
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 3177 into "/content/gdrive/My Drive/defteval/Bert/checkpoints/"/model.ckpt.


INFO:tensorflow:Saving checkpoints for 3177 into "/content/gdrive/My Drive/defteval/Bert/checkpoints/"/model.ckpt.


INFO:tensorflow:Loss for final step: 0.080251224.


INFO:tensorflow:Loss for final step: 0.080251224.


Training took time  0:06:36.573937


In [0]:
EVAL_BATCH_SIZE=10
eval_steps = int(len(test_features) / EVAL_BATCH_SIZE)
eval_input_fn = run_classifier.input_fn_builder(
      features=test_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)

In [0]:
result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Finished evaluation at 2019-12-04-13:53:48


INFO:tensorflow:Saving dict for global step 3177: auc = 0.7268537, eval_accuracy = 0.76628786, f1_score = 0.63512707, false_negatives = 347.0, false_positives = 270.0, global_step = 3177, loss = 0.91120124, precision = 0.6654275, recall = 0.60746604, true_negatives = 1486.0, true_positives = 537.0


INFO:tensorflow:Saving dict for global step 3177: auc = 0.7268537, eval_accuracy = 0.76628786, f1_score = 0.63512707, false_negatives = 347.0, false_positives = 270.0, global_step = 3177, loss = 0.91120124, precision = 0.6654275, recall = 0.60746604, true_negatives = 1486.0, true_positives = 537.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3177: "/content/gdrive/My Drive/defteval/Bert/checkpoints/"/model.ckpt-3177


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3177: "/content/gdrive/My Drive/defteval/Bert/checkpoints/"/model.ckpt-3177


In [0]:
print(result)

{'auc': 0.7268537, 'eval_accuracy': 0.76628786, 'f1_score': 0.63512707, 'false_negatives': 347.0, 'false_positives': 270.0, 'loss': 0.91120124, 'precision': 0.6654275, 'recall': 0.60746604, 'true_negatives': 1486.0, 'true_positives': 537.0, 'global_step': 3177}


In [0]:
result_train = estimator.evaluate(input_fn=train_input_fn, steps=num_train_steps)


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Finished evaluation at 2019-12-04-13:55:06


INFO:tensorflow:Saving dict for global step 3177: auc = 0.95250803, eval_accuracy = 0.95832545, f1_score = 0.9370243, false_negatives = 682.0, false_positives = 642.0, global_step = 3177, loss = 0.14741257, precision = 0.9388105, recall = 0.935245, true_negatives = 20596.0, true_positives = 9850.0


INFO:tensorflow:Saving dict for global step 3177: auc = 0.95250803, eval_accuracy = 0.95832545, f1_score = 0.9370243, false_negatives = 682.0, false_positives = 642.0, global_step = 3177, loss = 0.14741257, precision = 0.9388105, recall = 0.935245, true_negatives = 20596.0, true_positives = 9850.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3177: "/content/gdrive/My Drive/defteval/Bert/checkpoints/"/model.ckpt-3177


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3177: "/content/gdrive/My Drive/defteval/Bert/checkpoints/"/model.ckpt-3177


In [0]:
print(result_train)

{'auc': 0.95250803, 'eval_accuracy': 0.95832545, 'f1_score': 0.9370243, 'false_negatives': 682.0, 'false_positives': 642.0, 'loss': 0.14741257, 'precision': 0.9388105, 'recall': 0.935245, 'true_negatives': 20596.0, 'true_positives': 9850.0, 'global_step': 3177}
